# XGBoost (Regresión)

Además de las operaciones básicas de limpieza de datos, existen algunos requisitos para que XGBoost logre el máximo rendimiento. Principalmente:

* Las características numéricas deben escalarse
* Las características categóricas deben codificarse

En primer lugar, al igual que con cualquier otro conjunto de datos, importaremos el conjunto de datos. En este caso es el de Vivienda de Boston y lo almacenaremos en una variable llamada boston.

Importamos las librerías que vamos a utilizar

In [ ]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn import tree, datasets, metrics
#from sklearn import neighbors, datasets, metrics
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

Vamos a ignorar los warnings que no son importantes para lo que vamos a hacer. **Nota** En caso de no tener instalado XGBoost descomente y ejecuta  la línea de comando de instalación.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#pip install xgboost

## Entendimiento de los datos

Cargamos los datos para entenderlos

In [ ]:
print(housing.data.shape)
print(housing.DESCR)

In [ ]:
data = pd.DataFrame(housing.data)
data.columns = housing.feature_names

In [ ]:
data.head()

Resulta que este conjunto de datos tiene 8 columnas (incluida la variable de destino MedInc) y 20640 filas. Observe que las columnas son de tipo de datos flotantes, lo que indica la presencia de solo entidades continuas sin valores faltantes en ninguna de las columnas. 

In [ ]:
data.describe()

In [ ]:
data['median_house_value'] = housing.target

In [ ]:
data.info()

Separamos la variable objetivo y el resto de las variables usando .iloc para crear subconjuntos de datos.

In [ ]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]

Ahora, convertiremos el conjunto de datos en una estructura de datos optimizada llamada Dmatrix compatible con XGBoost y que le brinda su aclamado rendimiento  y ganancia de eficiencia.

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

Ahora, crearemos el conjunto de entrenamiento y prueba para la validación cruzada de los resultados utilizando la función train_test_split del módulo model_selection de sklearn con un tamaño de test_size igual al 20 % de los datos. Además, para mantener la reproducibilidad de los resultados, también se asigna un estado aleatorio.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

El siguiente paso es instanciar un objeto regresor XGBoost llamando a la clase XGBRegressor() de la biblioteca XGBoost con los hiperparámetros pasados como argumentos. Para problemas de clasificación, se habría utilizado la clase XGBClassifier().

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

Ajustamos el regresor al conjunto de entrenamiento y hacemos predicciones en el conjunto de prueba utilizando los métodos .fit() y .predict().

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

Calculamos el rmse invocando la función mean_sqaured_error del módulo de métricas de sklearn.

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

Crearemos un diccionario de hiperparámetros params que contiene todos los hiperparámetros y sus valores como pares clave-valor, pero excluiremos los n_estimadores del diccionario de hiperparámetros porque usaremos num_boost_rounds en su lugar.

Utilizaremos estos parámetros para crear un modelo de validación cruzada de 3 modelos, invocando el método cv() de XGBoost y almacenaremos los resultados en un dataframe cv_results. Tenga en cuenta que aquí está utilizando el objeto Dmatrix que creó anteriormente.

In [ ]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

Puede ver que su RMSE para la predicción de precios se ha reducido en comparación a la última  y resultó ser de alrededor de 0.633. Puede alcanzar un RMSE aún más bajo para un conjunto diferente de hiperparámetros. Puede considerar aplicar técnicas como **Grid Search, Random Search y Bayesian Optimization para alcanzar el conjunto óptimo de hiperparámetros.**

**Visualización de árboles de XGBoost y la importancia de sus atributos**

Podemos visualizar árboles individuales del modelo totalmente potenciado que crea XGBoost utilizando todo el conjunto de datos de viviendas. XGBoost tiene una función plot_tree() que facilita este tipo de visualización. Una vez que entrena un modelo usando la API de aprendizaje XGBoost, puede pasarlo a la función plot_tree() junto con la cantidad de árboles que desea trazar usando el argumento num_trees.

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

# Clasificación a partir de un ensamble extremo: XGBoost

Entrenaremos y construiremos nuestro modelo utilizando un conjunto de datos de clientes de supermercado.

In [ ]:
df = pd.read_csv("04-ComprasClientes.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

No hay valores faltantes. Ahora definimos la matriz de atributos y el vector de la variable objetivo.

In [ ]:
X = df.drop('Channel', axis=1)

y = df['Channel']

In [ ]:
X.head()

In [ ]:
y.head()

Podemos ver que las etiquetas y contienen valores como 1 y 2. Tendrmos que convertirlos en 0 y 1 para un análisis más detallado. Lo haremos de la siguiente manera

In [ ]:
# convert labels into binary values

y[y == 2] = 0

y[y == 1] = 1

In [ ]:
y.head()

Nuevamente creamos una estructura de datos optimizada con Dmatrix.

In [ ]:
# import XGBoost
import xgboost as xgb

# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
xgb_classifier = xgb.XGBClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

Dividimos los datos en conjuntos de prueba y entrenamiento. Luego con estos datos entrenaremos el clasificador XGBoost

In [ ]:
# import XGBClassifier
from xgboost import XGBClassifier

# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
                      
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)

# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

Haremos ahora las predicciones con XGBoost y calcularemos su exactitud.

In [ ]:
# make predictions on test data
y_pred = xgb_clf.predict(X_test)

In [ ]:
# check accuracy score
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

## Evaluación del modelo

Como antes podemos usar ciertos parámetros para construir un modelo de validación cruzada, llamando al método CV() de XGBoost.

In [ ]:
from xgboost import cv

params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [ ]:
xgb_cv.head()

In [ ]:
print((xgb_cv["test-auc-mean"]).tail(1))

Finalmente presentamos aquí también la importancia de las variables independientes

In [ ]:
xgb.plot_importance(xgb_clf)
plt.rcParams['figure.figsize'] = [6, 4]
plt.show()